# IS 455 - Data Visualization
## Programming Assignment 6 - Rashmi Chhabria

In [1]:
import pandas as pd
import bqplot
import numpy as np
import ipywidgets
from bqplot import pyplot as plt

In [2]:
building_inventory = pd.read_csv("building_inventory.csv")

In [3]:
# building_inventory.dtypes

In [4]:
building_inventory_map = building_inventory.groupby(['Congress Dist', 'Agency Name'])["Square Footage"].sum().unstack()

In [5]:
# building_inventory_map = building_inventory_map.replace(np.nan, 0)
# building_inventory_map = building_inventory_map.to_numpy()
# building_inventory_map

In [6]:
mySelectedLabel = ipywidgets.Label() # start with our label

In [7]:
def on_selected(change):
    if len(change['owner'].selected) == 1: #only 1 selected
        i, j = change['owner'].selected[0] # grab the x/y coordinates
        v = building_inventory_map.iloc[i,j] # grab data value at x/y index and mean along z
        row_val = building_inventory_map.index[i]
        col_val = building_inventory_map.columns[j]
        years = building_inventory.loc[(building_inventory['Agency Name'] == col_val) & (building_inventory['Congress Dist'] == row_val), 'Year Acquired'].values
        mySelectedLabel.value = 'Total Square Footage = ' + str(v) # set our label
        year_criteria = (building_inventory_by_year['Year Acquired'].isin(years))
#         line_chart = plt.clear()
        line_chart.x = building_inventory_by_year['Year Acquired'][year_criteria]
        line_chart.y = building_inventory_by_year['Square Footage'][year_criteria]

In [8]:
# (1) Scales: x/y, colors
col_sc = bqplot.ColorScale(scheme = "Blues")
x_sc = bqplot.LinearScale()
y_sc = bqplot.LinearScale()

# (2) Axis: x/y, colors
c_ax = bqplot.ColorAxis(scale = col_sc, 
                        orientation = 'vertical', 
                        side = 'right')

x_ax = bqplot.Axis(scale = x_sc, label = 'Agency Name')
y_ax = bqplot.Axis(scale = y_sc, 
                   orientation = 'vertical', label = 'Congress Dist')

# (3) Marks: heatmap
heat_map = bqplot.GridHeatMap(color = building_inventory_map,
                              scales = {'color': col_sc,
                                        'row': y_sc,
                                        'column': x_sc},
                              interactions = {'click': 'select'}, # make interactive on click of each box
                              anchor_style = {'fill':'pink'}, # to make our selection blue
                              selected_style = {'opacity': 1.0}, # make 100% opaque if box is selected
                              unselected_style = {'opacity': 0.8}) # make a little see-through if not

# (4) Link selection on heatmap to other things
heat_map.observe(on_selected, 'selected')

# (5) Paint heatmap canvas, don't display yet:
fig_heatmap = bqplot.Figure(marks = [heat_map], axes = [c_ax, y_ax, x_ax], title = 'Total Square Footage for Agency Name by Congressional Dist')

In [9]:
fig_heatmap.layout.min_width = '500px'

In [10]:
building_inventory_by_year = building_inventory[['Square Footage', 'Year Acquired']]
building_inventory_by_year = building_inventory_by_year.groupby(['Year Acquired']).sum()

In [11]:
year_more_than_zero = building_inventory_by_year.index > 0
building_inventory_by_year = building_inventory_by_year[year_more_than_zero]
building_inventory_by_year = building_inventory_by_year.reset_index()
# building_inventory_by_year

In [12]:
# (1) Scales: x/y
x_sc = bqplot.LinearScale(min = float(building_inventory_by_year['Year Acquired'].min()), max = float(building_inventory_by_year['Year Acquired'].max()))
y_sc = bqplot.LinearScale(min = float(building_inventory_by_year['Square Footage'].min()), max = float(building_inventory_by_year['Square Footage'].max()))

# (2) Axis: x/y
x_ax = bqplot.Axis(scale = x_sc, label = 'Square Footage')
y_ax = bqplot.Axis(scale = y_sc, 
                   orientation = 'vertical', label = 'Year Acquired')

# (3) Marks: line chart
line_chart = bqplot.Lines(x = building_inventory_by_year['Year Acquired'], y = building_inventory_by_year['Square Footage'],
                              scales = {'y': y_sc,
                                        'x': x_sc})

# (4) Paint line chart canvas, don't display yet:
fig = bqplot.Figure(marks = [line_chart], axes = [y_ax, x_ax], title = "Total Square Footage per Year")
fig.layout.min_width = '500px'
# fig

In [13]:
# fig = plt.figure(title="Total Square Footage per Year")
# axes_options = {'x': {'label': 'Year Acquired'}, 'y': {'label': 'Square Footage'}}
# line_chart = plt.plot(x = building_inventory_by_year.index, y = building_inventory_by_year['Square Footage'], axes_options=axes_options)
# fig.layout.min_width = '500px'
# line_chart.stroke_width = 2.5
# fig

In [14]:
# side by side figures
figures = ipywidgets.HBox([fig_heatmap, fig])

# label on top
myDashboard = ipywidgets.VBox([mySelectedLabel, figures])
myDashboard

As per the selection in heat map, the line plot updates itself to the years that match the selection criteria.
Keeping the x and y ranges static is done by specifying the min and max values which I have set as per the min and max values of the columns. The basic styling I have taken care of is matching the color schemes and specifying labels along with titles. There is definitely a lot more styling that can be done.